In [1]:
import pandas as pd
import numpy as np

In [2]:
metadata = pd.read_csv('isd-inventory.csv')

In [3]:
metadata.shape

(616681, 15)

In [4]:
metadata.head()

,USAF,WBAN,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,7005,99999,2012,18,0,0,0,0,0,0,0,0,0,0,0
1,7011,99999,2012,771,0,183,0,0,0,142,13,9,0,4,0
2,7018,99999,2013,0,0,0,0,0,0,710,0,0,0,0,0
3,7025,99999,2012,21,0,0,0,0,0,0,0,0,0,0,0
4,7026,99999,2012,0,0,0,0,0,0,367,0,0,0,0,7


In [5]:
usaf_wban_to_history = pd.read_csv('isd-history.csv')

In [6]:
usaf_wban_to_history.shape

(30053, 11)

In [7]:
usaf_wban_to_history.head()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,7005,99999,CWOS 07005,NaN,NaN,NaN,NaN,NaN,NaN,20120127,20120127
1,7011,99999,CWOS 07011,NaN,NaN,NaN,NaN,NaN,NaN,20120101,20121129
2,7018,99999,WXPOD 7018,NaN,NaN,NaN,0.0,0.0,7018.0,20130710,20130730
3,7025,99999,CWOS 07025,NaN,NaN,NaN,NaN,NaN,NaN,20120127,20120127
4,7026,99999,WXPOD 7026,AF,NaN,NaN,0.0,0.0,7026.0,20120713,20170822


In [8]:
# CTRY is displayed using the FIPS name in country.txt

In [9]:
china_files = usaf_wban_to_history[usaf_wban_to_history['CTRY'] == 'CH']

In [10]:
china_files.shape

(1333, 11)

So there are 1333 relevant files for china. Let's look at the relevant dates in these files.

In [11]:
import warnings; warnings.simplefilter('ignore')

In [12]:
china_files['start_year'] = china_files['BEGIN'].apply(lambda x: str(x)[:4])

In [13]:
china_files['end_year'] = china_files.END.apply(lambda x: str(x)[:4])

In [14]:
by_station_name = china_files.sort_values('STATION NAME', axis=0).reset_index()

I will choose nearby Chinese weather stations to get the data:

According to http://www.intellicast.com/Local/Weather.aspx?location=pek,

Weather stations nearby Beijing are: Chaoyang Qu, Xicheng Qu, FengSheng, Haidian Qu, and Tianning Temple

In the dataset, there is Fengxiang - 2005 to 2017

Also, Beijing's lat/long are 39.9N, 116 E - practically 40, 116.4:
according to https://www.latlong.net/

Got it: "BEIJING - CAPITAL INTERNATIONAL AIRPORT"

Haiyang Do is close: 39, 120 E
I did a search for 39 N, 117 E: got TIANJIN, BINHAI - both end in 2017


In [15]:
by_station_name.iloc[460:580,:];

In [16]:
china = china_files.fillna(0)

In [17]:
china['lat_int'] = china['LAT'].apply(lambda x: int(str(x).split('.')[0]))

In [18]:
china['lon_int'] = china['LON'].apply(lambda x: int(str(x).split('.')[0]))

In [19]:
china.loc[(china['lat_int'] == 40) & (china['lon_int'] == 116)]

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,start_year,end_year,lat_int,lon_int
11815,545110,99999,BEIJING - CAPITAL INTERNATIONAL AIRPORT,CH,0,ZBAA,40.08,116.585,35.4,19451031,20171116,1945,2017,40,116
29701,999999,44101,PEKING,CH,0,RPEK,40.00,116.500,50.9,19460430,19460630,1946,1946,40,116
